In [1]:
import os
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [2]:

import sys
import pandas as pd
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import torch.optim as optim
import numpy as np

In [3]:
train_df = pd.read_csv("/content/drive/Shareddrives/Datasets SEFAI/training_set.csv")
test_df = pd.read_csv("/content/drive/Shareddrives/Datasets SEFAI/test_set.csv")

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#Questo commento serve per provare il funzionamento dei commit con colab

batch_size = 1024

In [5]:
class CustomDataset(Dataset):
    def __init__(self, X_array, Y_array, transform=None):
        self.X = X_array
        self.Y = Y_array
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        image = self.X[index]
        label = self.Y[index]

        # Esegui le trasformazioni se definite
        if self.transform is not None:
            image = self.transform(image)

        return image, label

In [6]:

test_df['landmarks'] = test_df['landmarks'].apply(lambda lab: eval(lab))

train_df['landmarks'] = train_df['landmarks'].apply(lambda lab: eval(lab))
print(train_df)

                                      nome  \
0        train/angry/Training_12391352.jpg   
1        train/angry/Training_52407046.jpg   
2        train/angry/Training_20666200.jpg   
3        train/angry/Training_11533347.jpg   
4        train/angry/Training_60003551.jpg   
...                                    ...   
26700  train/disgust/Training_63164084.jpg   
26701  train/disgust/Training_25610374.jpg   
26702  train/disgust/Training_67023235.jpg   
26703  train/disgust/Training_61032772.jpg   
26704  train/disgust/Training_96306068.jpg   

                                               landmarks    label  
0      [[0.5751925706863403, 0.5731657147407532, -0.1...    angry  
1      [[0.43962034583091736, 0.7553703784942627, -0....    angry  
2      [[0.6029075384140015, 0.5643556118011475, -0.0...    angry  
3      [[0.5550940036773682, 0.7429446578025818, -0.0...    angry  
4      [[0.5235995054244995, 0.8211154937744141, -0.0...    angry  
...                                    

In [7]:
print(type(train_df.at[0,"landmarks"]))

label_dict = {"angry":0, "sad": 1, "neutral": 2, "surprise": 3, "disgust": 4, "fear": 5, "happy": 6}

test_df['label'] = test_df['label'].apply(lambda lab: label_dict[lab])

train_df['label'] = train_df['label'].apply(lambda lab: label_dict[lab])

array_train = train_df['landmarks'].to_numpy()
X_train = np.stack([np.array(lst) for lst in array_train])
y_train = train_df['label'].to_numpy()
array_test = test_df['landmarks'].to_numpy()
X_test = np.stack([np.array(lst) for lst in array_test])
y_test = test_df['label'].to_numpy()



<class 'list'>


In [8]:
train_dataset = CustomDataset(X_train, y_train, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


test_dataset = CustomDataset(X_test, y_test, transform=transforms.ToTensor())

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print( X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(26705, 478, 3) (26705,) (6678, 478, 3) (6678,)


In [9]:
def accuracy(preds, labels):
  probabilities = torch.nn.functional.softmax(preds, dim=1)
  _, predicted = torch.max(probabilities, dim=1)
  n_correct = (predicted==labels).sum().float()

  acc =n_correct / labels.shape[0]
  acc= torch.round(acc*100)
  return acc, n_correct;

In [10]:
class MultiClassificator(nn.Module):
  def __init__(self, in_size: int, hidden_size: int, num_classes: int):
    super(MultiClassificator, self).__init__()

    self.fc1 = nn.Linear(in_size, 1024, dtype=torch.float64)
    self.fc2 = nn.Linear(1024, 500, dtype=torch.float64)
    self.fc3 = nn.Linear(500, 100, dtype=torch.float64)
    self.fc4 = nn.Linear(100, num_classes, dtype=torch.float64)

  def forward(self, x: torch.Tensor):
    b = x.shape[0]
    x = x.view(b,-1)

    out = self.fc1(x)
    out = F.relu(out)
    out = self.fc2(out)
    out = F.relu(out)
    out = self.fc3(out)
    out = F.relu(out)
    out = self.fc4(out)
    return out

In [11]:
model = MultiClassificator(478*3, 1024,  7)
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

MultiClassificator(
  (fc1): Linear(in_features=1434, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=7, bias=True)
)


In [12]:
patience = 3

acc_list_train=[]
acc_list_test=[]


n_total_steps = len(train_loader)


best_loss = 100
counter=0
stop=False
num_epochs = 100
for epoch in range(num_epochs):
        model.train()
        print(counter)
        if stop:
          print(stop)
          break
        running_loss = 0.0
        running_acc = 0
        seen = 0
        for images, labels in train_loader:

          images = images.to(device)
          labels = labels.to(device)


          outputs = model(images)

          loss = criterion(outputs, labels)

          _, acc = accuracy(outputs, labels)
          seen +=labels.shape[0]

          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          running_loss += loss.item()
          running_acc += acc

        print (f'Epoch [{epoch}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Acc: {running_acc/seen:.4f}')
        acc_list_train.append(running_acc/len(train_loader))
        model.eval()

        tot_corrette = 0
        tot_eseguite = 0
        running_test_loss = 0
        val_loss = 0

        with torch.no_grad():

          for images, labels in test_loader:
              images = images.to(device)
              labels = labels.to(device)

              outputs = model(images)
              test_loss = criterion(outputs, labels)
              _, n_corrette=accuracy(outputs, labels)

              running_test_loss += test_loss.item()
              tot_corrette+=n_corrette.item()
              tot_eseguite+=labels.shape[0]

          test_acc=100* (tot_corrette/tot_eseguite)
          val_loss = running_test_loss / len(test_loader)
          acc_list_test.append(test_acc)
          print("Test acc: ", test_acc)
          print("Test loss: ", val_loss)


        if val_loss < best_loss:
          print("MIGLIORATO")
          torch.save(model.state_dict(), 'model_weights.pth')
          best_loss = val_loss
          best_model_train_acc=running_acc/seen
          best_model_test_acc=test_acc
          best_model_test_loss=val_loss
          best_model_train_loss=running_loss / len(train_loader)
          counter = 0
          # Salva i pesi del modello se la validation loss è migliorata
          torch.save(model.state_dict(), 'best_model.pt')
        else:
          counter += 1
        # Verifica se raggiunto il criterio di early stopping
          if counter >= patience:
              print(f'Early stopping at epoch {epoch+1}')
              stop=True
        print("BEST TEST LOSS: ", best_loss)

0
Epoch [0/100], Loss: 1.8179, Acc: 0.2557
Test acc:  25.666367175801135
Test loss:  1.7773912359270627
MIGLIORATO
BEST TEST LOSS:  1.7773912359270627
0
Epoch [1/100], Loss: 1.7841, Acc: 0.2611
Test acc:  25.666367175801135
Test loss:  1.7258684606116026
MIGLIORATO
BEST TEST LOSS:  1.7258684606116026
0
Epoch [2/100], Loss: 1.7175, Acc: 0.3054
Test acc:  37.61605271039233
Test loss:  1.558388263953232
MIGLIORATO
BEST TEST LOSS:  1.558388263953232
0
Epoch [3/100], Loss: 1.6005, Acc: 0.3685
Test acc:  40.491165019466905
Test loss:  1.5389430849449994
MIGLIORATO
BEST TEST LOSS:  1.5389430849449994
0
Epoch [4/100], Loss: 1.5340, Acc: 0.3995
Test acc:  42.15333932315065
Test loss:  1.4789878845635858
MIGLIORATO
BEST TEST LOSS:  1.4789878845635858
0
Epoch [5/100], Loss: 1.4805, Acc: 0.4231
Test acc:  42.54267744833783
Test loss:  1.411448215830996
MIGLIORATO
BEST TEST LOSS:  1.411448215830996
0
Epoch [6/100], Loss: 1.4826, Acc: 0.4260
Test acc:  41.85384845762204
Test loss:  1.417391519985991

In [13]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, classification_report

model = MultiClassificator(478*3, 1024,  7)
model.load_state_dict(torch.load("model_weights.pth"))
model.eval()
model.to(device)



# Calcola le metriche sul test dataset
model.eval()  # Imposta il modello in modalità di valutazione (non addestramento)
test_predictions = []
test_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predictions = torch.max(torch.nn.functional.softmax(outputs, dim=1), 1)
        test_predictions.extend(predictions.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(test_labels, test_predictions)
precision = precision_score(test_labels, test_predictions, average=None)
f1 = f1_score(test_labels, test_predictions, average=None)
#auc_roc = roc_auc_score(test_labels, test_predictions, multi_class='ovr')
classification_rep = classification_report(test_labels, test_predictions)

print("Test Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1 Score:", f1)
#print("AUC-ROC:", auc_roc)
print("Classification Report:\n", classification_rep)




Test Metrics:
Accuracy: 0.46211440551063193
Precision: [0.23924731 0.36801752 0.37774817 0.38711776 0.         0.25409836
 0.65741155]
F1 Score: [0.14530612 0.33267327 0.42218913 0.5133736  0.         0.05832549
 0.74385609]
Classification Report:
               precision    recall  f1-score   support

           0       0.24      0.10      0.15       853
           1       0.37      0.30      0.33      1107
           2       0.38      0.48      0.42      1185
           3       0.39      0.76      0.51       781
           4       0.00      0.00      0.00        97
           5       0.25      0.03      0.06       941
           6       0.66      0.86      0.74      1714

    accuracy                           0.46      6678
   macro avg       0.33      0.36      0.32      6678
weighted avg       0.41      0.46      0.41      6678



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

In [15]:
!pip install mediapipe

import zipfile


zip_ref = zipfile.ZipFile('/content/drive/Shareddrives/Datasets SEFAI/scraped_pictures.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall() #Extracts the files into the /tmp folder
zip_ref.close()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 20.7 MB/s eta 0:00:00


In [17]:
import mediapipe as mp
import glob, os
import cv2

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

label_dict = {"angry":0, "sad": 1, "neutral": 2, "surprise": 3, "disgust": 4, "fear": 5, "happy": 6}
label_arr=[]
images = []
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5) as face_mesh:

    for root, dirs, files in os.walk("photos"):
      for dir in dirs:
        if(not dir==".ipynb_checkpoints"):
          label=label_dict[dir]
          for _file in glob.glob("photos/"+dir+"/*.*"):
              image = cv2.imread(_file)
          # Convert the BGR image to RGB before processing.
              #print(_file)
              results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))


              annotated_image = image.copy()
              if(results.multi_face_landmarks!=None):
                for face_landmarks in results.multi_face_landmarks:
                  #print('face_landmarks:', face_landmarks)
                  mp_drawing.draw_landmarks(
                      image=annotated_image,
                      landmark_list=face_landmarks,
                      connections=mp_face_mesh.FACEMESH_TESSELATION,
                      landmark_drawing_spec=None,
                      connection_drawing_spec=mp_drawing_styles
                      .get_default_face_mesh_tesselation_style())
                  mp_drawing.draw_landmarks(
                      image=annotated_image,
                      landmark_list=face_landmarks,
                      connections=mp_face_mesh.FACEMESH_CONTOURS,
                      landmark_drawing_spec=None,
                      connection_drawing_spec=mp_drawing_styles
                      .get_default_face_mesh_contours_style())
                  mp_drawing.draw_landmarks(
                      image=annotated_image,
                      landmark_list=face_landmarks,
                      connections=mp_face_mesh.FACEMESH_IRISES,
                      landmark_drawing_spec=None,
                      connection_drawing_spec=mp_drawing_styles
                      .get_default_face_mesh_iris_connections_style())
                points_array = np.array(face_landmarks)

                x_array=[]
                y_array=[]
                z_array=[]

                landmark_matrix = []
                for data_point in face_landmarks.landmark:
                  landmark_matrix.append([data_point.x, data_point.y, data_point.z])

                label_arr.append(label)
                images.append(landmark_matrix)


In [18]:
images = np.array(images)
labels = np.array(label_arr)
scraped_dataset = CustomDataset(images, labels, transform=transforms.ToTensor())

scraped_loader = DataLoader(scraped_dataset, batch_size=batch_size, shuffle=False)

In [20]:
model = MultiClassificator(478*3, 1024,  7)
model.load_state_dict(torch.load("model_weights.pth"))
model.eval()
model.to(device)



# Calcola le metriche sul test dataset
model.eval()  # Imposta il modello in modalità di valutazione (non addestramento)
test_predictions = []
test_labels = []

with torch.no_grad():
    for images, labels in scraped_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predictions = torch.max(torch.nn.functional.softmax(outputs, dim=1), 1)
        test_predictions.extend(predictions.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(test_labels, test_predictions)
precision = precision_score(test_labels, test_predictions, average=None)
f1 = f1_score(test_labels, test_predictions, average=None)
#auc_roc = roc_auc_score(test_labels, test_predictions, multi_class='ovr')
classification_rep = classification_report(test_labels, test_predictions)

print("Test Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1 Score:", f1)
#print("AUC-ROC:", auc_roc)
print("Classification Report:\n", classification_rep)

Test Metrics:
Accuracy: 0.3079019073569482
Precision: [0.1625     0.21568627 0.39583333 0.24096386 0.         0.
 0.56818182]
F1 Score: [0.17931034 0.23655914 0.48717949 0.29850746 0.         0.
 0.6097561 ]
Classification Report:
               precision    recall  f1-score   support

           0       0.16      0.20      0.18        65
           1       0.22      0.26      0.24        42
           2       0.40      0.63      0.49        30
           3       0.24      0.39      0.30        51
           4       0.00      0.00      0.00        74
           5       0.00      0.00      0.00        29
           6       0.57      0.66      0.61        76

    accuracy                           0.31       367
   macro avg       0.23      0.31      0.26       367
weighted avg       0.24      0.31      0.27       367



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13